In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from collections import defaultdict
from json import JSONDecodeError
from qwikidata.sparql import return_sparql_query_results # python library for working with sparql and linked data from WikiData
import time
from requests.exceptions import ChunkedEncodingError
import math
#from SPARQLWrapper import SPARQLWrapper, JSON # sparql query library
import ssl

ModuleNotFoundError: No module named 'qwikidata'

# Explore csv

Meglio tenere il corso_codice perchè è la foreign key di DegreeProgrammes e CourseOfStudies (educational activities): serve per unire i due database per facilitare le relazioni successive. 

In [4]:
 # create first dataframe only using the specified columns 
dataDegree = pd.read_csv("DegreeProgrammes.csv", sep=",",
                    usecols=["annoaccademico", "corso_codice", "corso_descrizione", "url", "campus", "sededidattica", "ambiti", "tipologia", "durata", "internazionale", "internazionale_titolo", "accesso"], encoding="utf-8")

# .pickle is a python serialization format for easy and quick read-write, and pandas supports it natively
dataDegree.to_pickle("00_first_Degree.pickle")

# the first table we have looks like this:
dataDegree.head()

,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,accesso
0,659,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze
1,660,2018/2019,Giurisprudenza,http://www.unibo.it/it/didattica/corsi-di-stud...,Ravenna,Ravenna,Giurisprudenza,Laurea Magistrale a ciclo unico,5,False,NaN,libero accesso con verifica delle conoscenze
2,881,2018/2019,Architettura,http://www.unibo.it/it/didattica/corsi-di-stud...,Cesena,Cesena,Ingegneria e architettura,Laurea Magistrale a ciclo unico,5,False,NaN,numero chiuso
3,884,2018/2019,Chimica industriale,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Scienze,Laurea Magistrale,2,True,Titolo multiplo (solo alcuni curricula),libero accesso con verifica dell'adeguatezza d...
4,897,2018/2019,Direzione aziendale,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Economia e management,Laurea Magistrale,2,True,NaN,numero chiuso


In [45]:
#ora filtriamo il database originale selezionando solo l'ambito umanistico, quello che tratteremo nella nostra ontologia
#Poi, resettiamo l'index in modo che mi dia un nuovo ordine 

In [5]:
data_mask = dataDegree['ambiti']== "Studi umanistici"
dataHumanitiesDegree = dataDegree[data_mask].reset_index(drop=True)
dataHumanitiesDegree.head()

,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,accesso
0,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze
1,957,2018/2019,Filosofia,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze
2,962,2018/2019,Storia,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze
3,964,2018/2019,Antropologia culturale ed etnologia,http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea Magistrale,2,False,NaN,libero accesso con verifica dell'adeguatezza d...
4,966,2018/2019,"Cinema, televisione e produzione multimediale",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea Magistrale,2,False,NaN,libero accesso con verifica dell'adeguatezza d...


In [6]:
dataHumanitiesDegree.to_pickle("00_Humanities_Degree.pickle")

In [8]:
dataCourse = pd.read_csv("CoursesOfStudies.csv", sep=",",
                    usecols=["corso_codice", "materia_codice", "materia_descrizione"], encoding="utf-8")

dataCourse.to_pickle("00_first_Course.pickle")

dataCourse.head()

,corso_codice,materia_codice,materia_descrizione
0,8894,81644,LABORATORY ON RENEWABLE RAW MATERIALS AND CIRC...
1,9222,85238,BAYESIAN INFERENCE
2,8420,87462,HISTORY OF ECONOMIC THOUGHT
3,9222,85199,SURVEY SAMPLING
4,8412,87608,TECNOLOGIA E LEGISLAZIONE FARMACEUTICHE


In [9]:
dataHumanitiesDegree["corso_codice"] = dataHumanitiesDegree["corso_codice"].astype(str)
dataCourse["corso_codice"] = dataCourse["corso_codice"].astype(str)
data = pd.merge(dataHumanitiesDegree, dataCourse, on="corso_codice", how="left")

In [10]:
data["Italian University"] = "University of Bologna"

In [73]:
data.to_pickle("00_data.pickle")
pd.set_option("display.max_rows", None)
data.head(200)

,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,accesso,materia_codice,materia_descrizione,Italian University
0,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,13769,ACUSTICA MUSICALE (1),University of Bologna
1,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,27104,ANALISI DEL FILM,University of Bologna
2,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,45,ANTROPOLOGIA CULTURALE,University of Bologna
3,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,41822,CINEMATOGRAFIA DOCUMENTARIA E SPERIMENTALE (1),University of Bologna
4,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,87329,DIDATTICA DELLA STORIA DELL'ARTE (1),University of Bologna
5,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,3510,DRAMMATURGIA,University of Bologna
6,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,3522,DRAMMATURGIA MUSICALE,University of Bologna
7,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,78048,ELEMENTI DI ARMONIA E CONTRAPPUNTO,University of Bologna
8,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,345,ESTETICA,University of Bologna
9,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,3527,ETNOMUSICOLOGIA,University of Bologna


Ora, cerchiamo di incorporare i dati di un nuovo csv contenente gli SSD e i CFU degli insegnamenti, basandoci sul codice materia come chiave comune. 

In [11]:
SSDandCFU = pd.read_csv("DettagliCurriculaSSDeCFU.csv", sep=",",
                    usecols=["materia_codice", "insegnamento_ssd", "insegnamento_crediti"], encoding="utf-8")

SSDandCFU["materia_codice"] = SSDandCFU["materia_codice"].astype(str)
data["materia_codice"] = data["materia_codice"].astype(str)
SSDandCFU.head()

,materia_codice,insegnamento_ssd,insegnamento_crediti
0,222.0,IUS/08,12.0
1,74764.0,IUS/01,12.0
2,536.0,IUS/18,8.0
3,24331.0,IUS/19,8.0
4,230.0,IUS/13,9.0


In [12]:
first_column = data.pop('materia_codice')

data.insert(0, 'materia_codice', first_column)

In [13]:
finalData = data.merge(SSDandCFU, on="materia_codice", how="left")

In [115]:
finalData.head()

,materia_codice,corso_codice,annoaccademico,corso_descrizione,url,campus,sededidattica,ambiti,tipologia,durata,internazionale,internazionale_titolo,accesso,materia_descrizione,Italian University,insegnamento_ssd,insegnamento_crediti
0,13769,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,ACUSTICA MUSICALE (1),University of Bologna,NaN,NaN
1,27104,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,ANALISI DEL FILM,University of Bologna,NaN,NaN
2,45,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,ANTROPOLOGIA CULTURALE,University of Bologna,NaN,NaN
3,41822,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,CINEMATOGRAFIA DOCUMENTARIA E SPERIMENTALE (1),University of Bologna,NaN,NaN
4,87329,956,2018/2019,"Dams - discipline delle arti, della musica e ...",http://www.unibo.it/it/didattica/corsi-di-stud...,Bologna,Bologna,Studi umanistici,Laurea,3,False,NaN,libero accesso con verifica delle conoscenze,DIDATTICA DELLA STORIA DELL'ARTE (1),University of Bologna,NaN,NaN


In [29]:
#rename columns
FinalData.rename(columns = {'corso_codice':'DegreeUniversityID', 'annoaccademico':'academicYear'}, inplace = True)